### Connect to the database, PmP, on the MySQL server running on localhost

In [1]:
#import all the required modules
import pymysql
import pandas as pd
import warnings
import mysql.connector

In [2]:
#connect to PmP database using pymysql.connect
conn = pymysql.connect(host="localhost",user="root",password="",database="PmP")

### Explore the different tables and their columns

#### Table 1: Uniprot_to_Reactome
I sourced this table from Reactome, a manually-curated, reliable knowledgebase of biological pathways. This table lists all proteins and the pathways that they belong in. 

In [3]:
pd.read_sql_query("SELECT * FROM Uniprot_to_Reactome limit 3", conn)

,Uniprot_ID,Pathway_ID,Pathway_Link,Pathway_Name,Mis_Code,Species
0,A0A075B6P5,R-HSA-109582,https://reactome.org/PathwayBrowser/#/R-HSA-10...,Hemostasis,TAS,Homo sapiens
1,A0A075B6P5,R-HSA-1280218,https://reactome.org/PathwayBrowser/#/R-HSA-12...,Adaptive Immune System,TAS,Homo sapiens
2,A0A075B6P5,R-HSA-1280218,https://reactome.org/PathwayBrowser/#/R-HSA-12...,Adaptive Immune System,IEA,Homo sapiens


##### Removing duplicates of Uniprot_ID-Pathway pairs
There are some duplicates of Uniprot_ID-Pathway in the Uniprot_to_Reactome table due to different evidence codes (column Mis_Code) that is not of interest for this project, so I wrote a query to create a new table to remove these duplicates.  

In [ ]:
pd.read_sql_query("""CREATE TABLE Uniprot_to_Reactome_edited AS
                   SELECT DISTINCT Uniprot_ID, Pathway_ID, Pathway_Link, Pathway_Name FROM Uniprot_to_Reactome""", conn)

You can now see that the duplicate for the A0A075B6P5-R-HSA-1280218 pair has been removed in the new table, Uniprot_to_Reactome_edited.

In [4]:
pd.read_sql_query("SELECT * FROM Uniprot_to_Reactome_edited limit 3", conn)

,Uniprot_ID,Pathway_ID,Pathway_Link,Pathway_Name
0,A0A075B6P5,R-HSA-109582,https://reactome.org/PathwayBrowser/#/R-HSA-10...,Hemostasis
1,A0A075B6P5,R-HSA-1280218,https://reactome.org/PathwayBrowser/#/R-HSA-12...,Adaptive Immune System
2,A0A075B6P5,R-HSA-1643685,https://reactome.org/PathwayBrowser/#/R-HSA-16...,Disease


#### Table 2: Chemical_Tools
This table is a table of expert-reviewed chemical tools, compounds used in biochemical/cellular assays to study protein function. This table lists the chemical tools and the protein(s) they target. 

In [5]:
pd.read_sql_query("SELECT * FROM Chemical_Tools limit 3", conn)

,Probe_SMILES,Probe_Name,Target_Name,Uniprot_ID
0,C[C@@]1(CCN1C(c1csc2ccccc12)=O)C(N(CCCC(O)=O)C...,GPCR_GLPG-0974/CPP_GLPG0974,FFAR2,O15552
1,C1CN(CCN1Cc1nc2ccccc2[nH]1)c1ccccn1,GPCR_ABT-724/CPP_ABT-724,DRD4,P21917
2,COc1cc(cc(c1OC)OC)Nc1ccnc2ccc(cc12)[Br],KCGS_SGC-GAK-1/EUb_SGC-GAK-1/CPP_SGC-GAK-1,GAK,O14976


### TASK 1: Show mapped results for a pathway of interest
I wrote a query that would result in a table listing all proteins that pertain in the pathway of interest and chemical tools that target them (if any). Each row is unique for a protein-chemical_tool pair, and proteins that do not have chemical tools are also included in the list with the Probe_Name and Probe_SMILES values being NULL (i.e. 'None').

The example pathway I used is the 'Interleukin-6 signaling' pathway (Pathway_ID: R-HSA-1059683).

In [6]:
pd.read_sql_query("""SELECT U.Pathway_ID, U.Pathway_Name, U.Uniprot_ID, C.Probe_Name, C.Probe_SMILES
                     FROM Uniprot_to_Reactome_edited U LEFT JOIN Chemical_Tools C 
                         ON(U.Uniprot_ID = C.Uniprot_ID) 
                     WHERE U.Pathway_ID='R-HSA-1059683'""", conn)

,Pathway_ID,Pathway_Name,Uniprot_ID,Probe_Name,Probe_SMILES
0,R-HSA-1059683,Interleukin-6 signaling,O14543,None,None
1,R-HSA-1059683,Interleukin-6 signaling,O60674,CPP_Ruxolitinib,C1CCC(C1)[C@@H](CC#N)n1cc(cn1)c1c2cc[nH]c2ncn1
2,R-HSA-1059683,Interleukin-6 signaling,O60674,CPP_Gandotinib,Cc1c(Cc2ccc(cc2F)[Cl])n2c(c(CN3CCOCC3)cc(Nc3cc...
3,R-HSA-1059683,Interleukin-6 signaling,O60674,CPP_BMS-911543,CCn1c(cc2c3c(c(Nc4cc(C)n(C)n4)nc12)ncn3C)C(N(C...
4,R-HSA-1059683,Interleukin-6 signaling,O60674,CPP_AZ-960,Cc1cc(Nc2c(cc(C#N)c(N[C@@H](C)c3ccc(cc3)F)n2)F...
5,R-HSA-1059683,Interleukin-6 signaling,O60674,KCGS_BI01125081,C1COCCN1c1ccc(cn1)c1cc(c2cccnc2c1)OCc1c(cccn1)...
6,R-HSA-1059683,Interleukin-6 signaling,O60674,KCGS_AKI-007a,CCCNC(c1ccc(cc1)Nc1nc(c2c(cco2)n1)NCC(F)(F)F)=O
7,R-HSA-1059683,Interleukin-6 signaling,O60674,KCGS_GSK1379761,COc1cc(cc(c1OC)OC)Nc1nccc(n1)N1CCC(C1)NC(C1COc...
8,R-HSA-1059683,Interleukin-6 signaling,O60674,KCGS_PFE-PKIS 34,C[C@@H]1CCN(C[C@@H]1N(C)c1c2cc[nH]c2ncn1)C(CO)=O
9,R-HSA-1059683,Interleukin-6 signaling,O60674,KCGS_ESD0001927,CC[C@@H](CO)Nc1c(cnc(Nc2cccc(c2)NC(C(C)(C)C)=O...


### TASK 2: Get the chemical coverage (i.e. percentage of proteins that have chemical tools) for a pathway of interest 

Perhaps one is interested in determining whether more efforts for chemical tool development is required for the pathway of interest. This can be gauged by determining how many, or what percent, of the proteins that belong in the pathway have chemical tools that target them. 

Here is the query I wrote to determine the chemical coverage of the 'Interleukin-6 signaling' pathway (Pathway_ID: R-HSA-1059683).

In [7]:
pd.read_sql_query("""
                  SELECT x.number / y.number as Chemical_Coverage
                  FROM
                  (
                  SELECT COUNT(*) as number
                  FROM Uniprot_to_Reactome_edited U LEFT JOIN Chemical_Tools C 
                      ON(U.Uniprot_ID = C.Uniprot_ID) 
                  WHERE U.Pathway_ID='R-HSA-1059683' and C.Probe_Name IS NULL
                  ) x
                  JOIN
                  (
                  SELECT COUNT(DISTINCT U.Uniprot_ID) as number
                  FROM Uniprot_to_Reactome_edited U LEFT JOIN Chemical_Tools C 
                      ON(U.Uniprot_ID = C.Uniprot_ID) 
                  WHERE U.Pathway_ID='R-HSA-1059683'
                  ) y
                  """, conn)

,Chemical_Coverage
0,0.5455
